In [6]:
import re
import os
import subprocess
import subprocess


In [7]:
import re

def extract_parameters(makefile_content):
    """
    Extract parameters from Makefile content and return a dictionary.

    Args:
    - makefile_content (str): Text content of a Makefile.

    Returns:
    - dict: Dictionary with parameter names as keys and their values.
    """
    parameters = {}
    # Pattern to match parameter definitions including string values
    pattern = re.compile(r'-P \S+\.(\w+)=(?:"([^"]+)"|(\d+))')

    for line in makefile_content.splitlines():
        if match := pattern.search(line):
            param_name = match[1]
            # The parameter value could be a string (group 2) or a number (group 3)
            param_value = match[2] if match[2] else int(match[3])
            parameters[param_name] = param_value

    return parameters

In [8]:


def convert_to_pytest(unit, makefile_path, testbench_path):
    with open(makefile_path, 'r') as file:
        makefile_content = file.read()

    # Extract relevant details from the Makefile
    verilog_sources = re.findall(r"\(\s*REPO_ROOT\s*\)/rtl/common/\s*(\S+\.sv)", makefile_content)
    dut = re.search(r"DUT\s*=\s*(\w+)", makefile_content).group(1)

    # Extract parameters and values
    params = extract_parameters(makefile_content)
    param_names_str = ','.join(params.keys())
    # Extract parameter values and format them in the desired way
    param_values_str = str([tuple(params.values())])
    # Create a comma-separated string of lower case parameter names
    param_names_lower_str = ', '.join([key.lower() for key in params.keys()])


    # Read the testbench.py and convert it to a pytest-compatible format
    with open(testbench_path, 'r') as tb_file:
        testbench_content = tb_file.read()

    # print(f'{verilog_sources=}')
    # print(f'{dut=}')
    # print(f'{params=}')
    # print(f'{param_names_str=}')
    output_str = f'''

repo_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode('utf-8')
tests_dir = os.path.abspath(os.path.dirname(__file__)) #gives the path to the test(current) directory in which this test.py file is placed
rtl_dir = os.path.abspath(os.path.join(repo_root, 'rtl/', 'common')) #path to hdl folder where .v files are placed

@pytest.mark.parametrize("{param_names_lower_str}", {param_values_str})
def test_{unit}(request, {param_names_lower_str}):
    dut = "{dut}"
    module = os.path.splitext(os.path.basename(__file__))[0]  # The name of this file
    toplevel = "{dut}"   

    verilog_sources = [
'''
    for file in verilog_sources:
        output_str += f'        os.path.join(rtl_dir, "{file}"),\n'
    output_str += '''
    ]
'''

    output_str += '    parameters = {'
    for key in params:
        output_str += f"'{key}':{key.lower()},"
    output_str += ' }\n'
    output_str += '''
    extra_env = {f'PARAM_{k}': str(v) for k, v in parameters.items()}

    # sourcery skip: no-conditionals-in-tests
    if request.config.getoption("--regression"):
        sim_build = os.path.join('regression_area', 'sim_build', request.node.name.replace('[', '-').replace(']', ''))
    else:
        sim_build = os.path.join('local_sim_build', request.node.name.replace('[', '-').replace(']', ''))

    run(
        python_search=[tests_dir],  # where to search for all the python test files
        verilog_sources=verilog_sources,
        toplevel=toplevel,
        module=module,
        parameters=parameters,
        sim_build=sim_build,
        extra_env=extra_env,
        waves=True,
    )
'''
    return testbench_content + output_str

In [9]:
# Define paths
repo_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode('utf-8')
src_dir = f'{repo_root}/val/common_level1'
dst_dir = f'{repo_root}/val/integration' 
units = [unit for unit in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir, unit))]
write_new_unit = True

FileNotFoundError: [Errno 2] No such file or directory: '/home/sean/github/RTLDesignSherpa/val/common_level1'

In [ ]:


for unit in units:
    makefile_path = os.path.join(src_dir, unit, "Makefile")
    testbench_path = os.path.join(src_dir, unit, "testbench.py")
    new_test_path = os.path.join(dst_dir, f"test_{unit}.py")

    testbench_content = convert_to_pytest(unit, makefile_path, testbench_path)

    if write_new_unit:
        # Write the new test_<unit>.py file
        with open(new_test_path, 'w') as new_test_file:
            new_test_file.write(testbench_content)
    else:
        print(testbench_content)

    print(f"Converted {unit} to pytest-compatible format.")

Converted arbiter_round_robin_wrapper to pytest-compatible format.
Converted arbiter_weighted_round_robin_wrapper to pytest-compatible format.


In [ ]:
import os
import re

# Define the directory containing the Python files.
directory = '/home/sean/github/RTLDesignSherpa/val/unit/'

# Regular expression pattern to find the old logging setup.
pattern = re.compile(r"def configure_logging\(dut_name, log_file_path\):.*?return log\n", re.DOTALL)

# Replacement text using the configure_logging function.
replacement = """
def configure_logging(dut_name, log_file_path):
    log = logging.getLogger(f'cocotb_log_{dut_name}')
    log.setLevel(logging.DEBUG)
    fh = logging.FileHandler(log_file_path)
    fh.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    log.addHandler(fh)
    return log
"""

# Iterate over the files in the directory.
for filename in os.listdir(directory):
    if filename.endswith('.py'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r+') as file:
            content = file.read()
            # Check if the file contains the old logging setup.
            if pattern.search(content):
                # Replace the old logging setup with the new function call.
                updated_content = pattern.sub(replacement, content)
                # Write the updated content back to the file.
                file.seek(0)
                file.write(updated_content)
                file.truncate()
